### Combining GDP data with the jobs report

Brian Dew

Updated: September 20, 2020

----

Notes:

The basic idea here is to use the BEA population estimates plus CPS employment rate and hours worked trends to estimate GDP per hour of work. BLS does this process for its productivity and costs report, using much more comprehensive data, however this approximation has proven decent over time.

The hard part to this measure is getting hours worked right. All of the published measures are either too-low-frequency or don't have a broad enough definition of workers. I want to capture all workers, regardless of full-time or part-time status or of whether they work for the private sector. Also want to capture second and third jobs. As a result, I selected total actual hours worked from the CPS microdata, specifically finding the trend using x13as with default settings. Hours worked from the CPS microdata have issues around holidays falling in the reference week, and also do not capture hours worked for some important categories of labor such as self-employed persons.

In [1]:
%matplotlib inline
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

qtrs = {1: 1, 2: 1, 3: 1, 4: 2, 5:2, 6:2, 7:3, 8:3, 9:3, 10:4, 11:4, 12:4}

In [2]:
avghrs = lambda x: np.average(x.HRSACTT.replace(-1, 0), weights=x.WGT)
aah, epop = {}, {}
cols = ['HRSACTT', 'LFS', 'MONTH']
for year in range(1989, 2024):
    wgt = 'PWSSWGT' if year >= 1998 else 'BASICWGT'
    df = pd.read_feather(cps_dir / f'cps{year}.ft', 
                         columns=cols + [wgt]).rename({wgt: 'WGT'}, axis=1)
    emp = df.query('LFS == "Employed"')
    ah = (emp.groupby('MONTH').apply(avghrs))
    aah.update({pd.to_datetime(f'{year}-{month}-01'): value 
                for month, value in list(zip(ah.index, ah.values))})
    ep = (emp.groupby('MONTH').WGT.sum() / df.groupby('MONTH').WGT.sum())
    epop.update({pd.to_datetime(f'{year}-{month}-01'): value 
                 for month, value in list(zip(ep.index, ep.values))})

In [3]:
sm = x13_arima_analysis(pd.Series(epop))
epop_sa = sm.seasadj.resample('QS').mean()
sm = x13_arima_analysis(pd.Series(aah))
aah_sa = sm.trend.resample('QS').mean()

/home/brian/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/x13.py:203: X13Warning: WARNING: Visually significant seasonal and trading day peaks have 
          been found in the estimated spectrum of the regARIMA residuals.
  
          found in one or more of the estimated spectra.
  warn(errors, X13Warning)


In [4]:
gdp_code = ('T10106', ['A191RX'])
pop_code = ('T70100', ['B230RC'])
cd = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'].iloc[-1]
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
gdp = rgdp / rgdp.iloc[-1] * cd
pop = nipa_df(retrieve_table(pop_code[0])['Data'], pop_code[1]).sort_index()

df = pd.DataFrame()
df['epop'] = epop_sa * 100
df['pop'] = pop['B230RC'] / 1000
#df['hours'] = fred_df('PRS84006023')
df['hours'] = aah_sa
df['gdp'] = gdp['A191RX']
df['input'] = (df['pop'] * (df['epop'] / 100)) * (df['hours'] * 52)
df['gdpinp'] = df['gdp'] / df['input']
datetxt = dtxt(df['gdpinp'].dropna().index[-1])['qtr1']
write_txt(text_dir / 'gdpjobs_ltdt.txt', datetxt)
df.to_csv(data_dir / 'gdpjobslvl.csv', index_label='date')

df['epop_c'] = ((df['pop'] * df['epop'].mean()) * df['hours']) * df['gdpinp']
df['pop_c'] = ((df['pop'].mean() * df['epop']) * df['hours']) * df['gdpinp']
df['hours_c'] = ((df['pop'] * df['epop']) * df['hours'].mean()) * df['gdpinp']

df_g = growth_rate(df.dropna())

df_g['pop_contr'] = df_g['gdp'] - df_g['pop_c']
df_g['epop_contr'] = df_g['gdp'] - df_g['epop_c']
df_g['hours_contr'] = df_g['gdp'] - df_g['hours_c']
df_g['prod'] = (df_g['gdp'] - df_g['pop_contr'] - 
                df_g['epop_contr'] - df_g['hours_contr'])
result = df_g[['pop_contr', 'epop_contr', 'hours_contr', 'prod']].round(2)
result.to_csv(data_dir / 'gdpjobs.csv', index_label='date')

In [5]:
data = df.dropna()

ltval = f"\${data['gdpinp'].iloc[-1]:.2f}"
prval = f"\${data['gdpinp'].iloc[-2]:.2f}"
prdate = dtxt(data['gdpinp'].index[-2])['qtr1']
prdt = '2019-10-01'
prval2 = f"\${data.loc[prdt, 'gdpinp']:.2f}"
prval3 = f"\${data.loc['2015-10-01', 'gdpinp']:.2f}"
val89 = f"\${data.loc['1989-01-01', 'gdpinp']:.2f}"
gdpval = f"\${data['gdp'].iloc[-1] / 1_000:,.0f}"
gdppr = f"\${data.loc[prdt, 'gdp'] / 1_000:,.0f}"
agghrs = f"{data['input'].iloc[-1] / 1_000:,.0f}"
hrspr = f"{data.loc[prdt, 'input'] / 1_000:,.0f}"

text = (f'In {datetxt}, real GDP was equivalent to roughly {ltval} '+
        f'per hour of work, compared to {prval} in {prdate}, {prval2} '+
        f'in 2019 Q4, {prval3} in 2015 Q4, and {val89} in the first quarter '+
        'of 1989.\n\nComparing the latest data to the pre-COVID data covering '+
        f'2019 Q4, annualized real GDP is {gdpval} billion in the latest data '+
        f'and {gdppr} billion in 2019 Q4. Aggregate hours worked total '+
        f'{agghrs} billion in the latest quarter and {hrspr} billion '+
        'in 2019 Q4.')
write_txt(text_dir / 'gdp_per_hour.txt', text)
print(text, '\n\n')

poplt = value_text(result['pop_contr'].iloc[-1], style='contribution_to', 
                   ptype='pp', digits=2)
poppr = value_text(result.loc[prdt, 'pop_contr'], style='contribution', 
                   ptype='pp', digits=2, casual=True)
emplt = value_text(result['epop_contr'].iloc[-1], style='contribution', 
                   ptype='pp', digits=2)
emppr = value_text(result.loc[prdt, 'epop_contr'], style='contribution', 
                   ptype='pp', digits=2, casual=True)
hrslt = value_text(result['hours_contr'].iloc[-1], style='contribution_to', 
                   ptype='pp', digits=2, casual=True)
hrspr = value_text(result.loc[prdt, 'hours_contr'], style='contribution', 
                   ptype='pp', digits=2, casual=True)
prodlt = value_text(result['prod'].iloc[-1], style='contribution_to', 
                    ptype='pp', digits=2)
prodpr = value_text(result.loc[prdt, 'prod'], style='contribution_of', 
                    ptype='pp', digits=2, casual=True)

text = (f'In {datetxt}, population growth {poplt} annualized GDP growth, '+
        f'and, for comparison, {poppr} in 2019 Q4. Changes in the '+
        f'employed share of the population {emplt} in the latest quarter, '+
        f'and {emppr} in the fourth quarter of 2019. Changes in average '+
        f'hours worked {hrslt} GDP growth in the latest quarter and {hrspr} '+
        f'in 2019 Q4. Lastly, productivity {prodlt} GDP growth in {datetxt}, '+
        f'compared to {prodpr} in 2019 Q4.')
write_txt(text_dir / 'gdpjobsch.txt', text)
print(text)

In 2022 Q4, real GDP was equivalent to roughly \$83.39 per hour of work, compared to \$82.98 in 2022 Q3, \$78.05 in 2019 Q4, \$75.77 in 2015 Q4, and \$50.76 in the first quarter of 1989.

Comparing the latest data to the pre-COVID data covering 2019 Q4, annualized real GDP is \$26,145 billion in the latest data and \$24,886 billion in 2019 Q4. Aggregate hours worked total 314 billion in the latest quarter and 319 billion in 2019 Q4. 


In 2022 Q4, population growth contributed 0.59 percentage point to annualized GDP growth, and, for comparison, added 0.59 percentage point in 2019 Q4. Changes in the employed share of the population contributed 0.38 percentage point in the latest quarter, and added 1.62 percentage points in the fourth quarter of 2019. Changes in average hours worked subtracted 0.29 percentage point from GDP growth in the latest quarter and added 0.37 percentage point in 2019 Q4. Lastly, productivity contributed 2.00 percentage points to GDP growth in 2022 Q4, compared to

In [6]:
df

,epop,pop,hours,gdp,input,gdpinp,epop_c,pop_c,hours_c
1989-01-01,48.433198,246.460,37.432207,1.179494e+07,232347.691393,50.764187,2.244419e+07,2.719979e+07,2.266310e+07
1989-04-01,48.388751,247.017,37.458860,1.188497e+07,232824.754160,51.046841,2.263627e+07,2.734559e+07,2.281983e+07
1989-07-01,48.467951,247.698,37.481782,1.197302e+07,233991.850775,51.168540,2.276672e+07,2.747245e+07,2.297484e+07
1989-10-01,48.425322,248.374,37.485017,1.199661e+07,234444.312997,51.170421,2.283166e+07,2.745167e+07,2.301812e+07
1990-01-01,48.348861,248.936,37.479501,1.212771e+07,234569.255754,51.702056,2.311766e+07,2.768901e+07,2.327309e+07
...,...,...,...,...,...,...,...,...,...
2022-01-01,48.266437,332.978,37.136392,2.580382e+07,310358.631991,83.141931,4.927084e+07,4.404378e+07,4.997504e+07
2022-04-01,48.514757,333.321,37.117055,2.576649e+07,312114.098154,82.554729,4.894775e+07,4.393482e+07,4.992876e+07
2022-07-01,48.639145,333.799,37.075754,2.597292e+07,313014.388331,82.976775,4.921372e+07,4.422339e+07,5.038483e+07
2022-10-01,48.683726,334.282,37.049253,2.614496e+07,313530.359815,83.388913,4.949432e+07,4.445198e+07,5.075483e+07


In [7]:
#growth_rate(df['gdpinp'].dropna()).iloc[-12:].plot(kind='bar')